<a href="https://colab.research.google.com/github/tien2612/Plant-Disease-Website/blob/master/model/apple_leaf_diseases_with_inceptionresnetv2_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TO RUN THIS NOTE BOOK
1. Download 2 dataset and pretrained model in Datasource from [Kaggle](https://www.kaggle.com/code/arnabs007/apple-leaf-diseases-with-inceptionresnetv2-keras/input)

2. Extract then change the 3 folders name to:
  plant-pathology-2021-fgvc8
  keras-pretrained-model
  resized_plant2021

3. Then put them in a folder name X

4. Change **your_path variable** below to your path to inside folder X


# What is this about?
Apples are one of the most important temperate fruit crops in the world. Foliar (leaf) diseases pose a major threat to the overall productivity and quality of apple orchards. The current process for disease diagnosis in apple orchards is based on manual scouting by humans, which is time-consuming and expensive. 

The main objective of the competition is to develop machine learning-based models to accurately classify a given leaf image from the test dataset to a particular disease category, and to identify an individual disease from multiple disease symptoms on a single leaf image. 

# Import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm import tqdm
import tensorflow_addons as tfa
import random
from sklearn.preprocessing import MultiLabelBinarizer

pd.set_option("display.max_columns", None)

pd.set_option("display.max_columns", None)

**Your path into folder contain datasets and pretrained model**

In [ ]:
your_path ='Documents/multidiscipline-project'

Let's explore the data.
How many images are in the datset, the labels and their frequencies.

In [ ]:
train = pd.read_csv(your_path + '/plant-pathology-2021-fgvc8/train.csv')
print(len(train))
print(train.columns)
# print(train['labels'].value_counts())
print(train['labels'].value_counts().plot.bar())

# Wait
Notice something. Unlike the previous year's competition [Plant Pathology 2020 - FGVC7](https://www.kaggle.com/c/plant-pathology-2020-fgvc7) one image can belong to more than one disease. Notice how in case of multiple diseases the diseases are seperated by an space. So there are not 12 labels, its actually just 6 labels.
5 dieseases: 

1. rust
2. scab 
3. complex 
4. frog eye leaf spot
5. powdery mildew 

and another label is 

6. healthy (healthy leaves) 

Now the most important thing is, as one image can have multiple diseases, that means this problem is **Multi label classification** problem. Many get confused betweeen multilabel and multiclass classification. if you are new to multilabel classification I would suggest going over this [An introduction to MultiLabel classification](https://www.geeksforgeeks.org/an-introduction-to-multilabel-classification/) . 

So now we gotta process the labels. And then lets find out the actual frequencies of the labels. 


In [ ]:
train['labels'] = train['labels'].apply(lambda string: string.split(' '))
train

First I convert the labels representation into **one hot encoded format** using MultilabelBinarizer from sklearn. Now we can see and plot the frequencies of each label. 

In [ ]:
s = list(train['labels'])
mlb = MultiLabelBinarizer()
trainx = pd.DataFrame(mlb.fit_transform(s), columns=mlb.classes_, index=train.index)
print(trainx.columns)
print(trainx.sum())

labels = list(trainx.sum().keys())
print(labels)
label_counts = trainx.sum().values.tolist()

fig, ax = plt.subplots(1,1, figsize=(20,6))

sns.barplot(x= labels, y= label_counts, ax=ax)

# Let's view some of the images

In [ ]:
fig1 = plt.figure(figsize=(26,10))

for i in range(1, 13):
    
    rand =  random.randrange(1, 18000)
    sample = os.path.join(your_path + '/plant-pathology-2021-fgvc8/train_images/', train['image'][rand])
    
    img = PIL.Image.open(sample)
    
    ax = fig1.add_subplot(4,3,i)
    ax.imshow(img)
    
    title = f"{train['labels'][rand]}{img.size}"
    plt.title(title)
    
    fig1.tight_layout()


# Imaze Size & Processing
from the titles we can see some random image sizes - (4000, 2672). Larger images are harder to process hence takes much longer to train the CNN. Downsampling all these 18632 images is also a time consuming task. This is I am going to use the resized imaged for this dataset [resized-plant2021](https://www.kaggle.com/ankursingh12/resized-plant2021) by Ankur Singh. He has already downsampled the images into size of 256, 384, 512 & 640px.

There are 18632 images in the training set. Even after using the downsampled images we cant fit all of the images into memory at once. So I have used the flow_from_dataframe method from keras. This method reads images in batch size from the storage without loading all the images at once and saving us from **GPU Out of Memory (OOM)** issue. 

In [ ]:
%%time
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                        preprocessing_function=None,
                                                        data_format=None,
                                                    )

# dir = '/home/vh/Documents/multidiscipline-project/resized_plant2021/img_sz_512'

dir = your_path + '/resized_plant2021/img_sz_512'

train_data = datagen.flow_from_dataframe(
    train,
    directory= dir,
    x_col="image",
    y_col= 'labels',
    color_mode="rgb",
    target_size = (256,256),
    class_mode="categorical",
    batch_size=32,
    shuffle=False,
    seed=40,
)
# print(directory)

# Transfer Learning
Transfer learning is the process of using frozen weights from a large pre-trained model for a downstream task which is in our case classifying leaf diseases. As we can't use internet in this notebook, I will use the dataset of keras's pretrained models containing the weights of 'imagenet'. The output/top layer of a pretrained layer is a dense layer containing number of nodes = number of output classes. All the models here are pre-trained on 'imagenet' hence they have a output/top layer of 1000 nodes. We will have to replace the output/top layer with our own dense layer with 6 nodes (for 6 classes). 

I am going to be using **Inception ResNet v2**.


In [ ]:
seed = 1200
tf.random.set_seed(seed)

weights_path = your_path + '/keras-pretrained-model/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'

model = keras.applications.InceptionResNetV2(weights=weights_path, include_top=False, input_shape=(256, 256, 3))

print(model.input)
print(model.output)

# Activation, Losses & Metrices

As this is a multilabel classification problem, we can't use softmax here, hence the sigmoid activation.

Binary crossentropy is used instead of categorical crossentropy. We use categorical cross-entropy in multi-class problems, but for multi-label problems, we use binary cross-entropy. Think of it this way, an image may have multiple labels, and we need the probabilities that each of these labels corresponds to the given image - this can be considered as n independent binary classifiers for the n labels.

For evaluation I have used F1 accuracy metrics instead of binary accuracy. F1 and its variants are better for evaluation when it comes to multiclass and multilabel problems. if you want to know F1 score works for Multilabel classification go through this https://medium.com/synthesio-engineering/precision-accuracy-and-f1-score-for-multi-label-classification-34ac6bdfb404https://medium.com/synthesio-engineering/precision-accuracy-and-f1-score-for-multi-label-classification-34ac6bdfb404. 

In [ ]:
new_model = tf.keras.Sequential([
    model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(6, 
        kernel_initializer=keras.initializers.RandomUniform(seed=seed),
        bias_initializer=keras.initializers.Zeros(), name='dense_top', activation='sigmoid')
])

# Freezing the weights
for layer in new_model.layers[:-1]:
    layer.trainable=False
    
new_model.summary()

In [ ]:
f1 = tfa.metrics.F1Score(num_classes=6, average='macro')

callbacks = keras.callbacks.EarlyStopping(monitor=f1, patience=3, mode='max', restore_best_weights=True)


new_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(learning_rate=1e-4), 
              metrics= [f1])

new_model.fit(train_data, epochs=40, callbacks=callbacks)

# Submission

For submission I will resize the test images and then predict the labels for them.

In [ ]:
test = pd.read_csv(your_path + '/plant-pathology-2021-fgvc8/sample_submission.csv')
for img_name in tqdm(test['image']):
    path = your_path + '/plant-pathology-2021-fgvc8/test_images/'+str(img_name)
    with PIL.Image.open(path) as img:
        img = img.resize((256,256))
        img.save(f'./{img_name}')

In [ ]:
test_data = datagen.flow_from_dataframe(
    test,
    directory = './',
    x_col="image",
    y_col= None,
    color_mode="rgb",
    target_size = (256,256),
    classes=None,
    class_mode=None,
    batch_size=32,
    shuffle=False,
    seed=40,
)

preds = new_model.predict(test_data)
print(preds)
preds = preds.tolist()

indices = []
for pred in preds:
    temp = []
    for category in pred:
        if category>=0.3:
            temp.append(pred.index(category))
    if temp!=[]:
        indices.append(temp)
    else:
        temp.append(np.argmax(pred))
        indices.append(temp)
    
print(indices)

In [ ]:
labels = (train_data.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

testlabels = []


for image in indices:
    temp = []
    for i in image:
        temp.append(str(labels[i]))
    testlabels.append(' '.join(temp))

print(testlabels)

Remove the resized images from output before submission. if there are any other files present except 'submission.csv' it will throw an error when submitting.

In [ ]:
delfiles = tf.io.gfile.glob('./*.jpg')

for file in delfiles:
    os.remove(file)

In [ ]:
sub = pd.read_csv(your_path + '/plant-pathology-2021-fgvc8/sample_submission.csv')
sub['labels'] = testlabels
sub.to_csv('submission.csv', index=False)
sub
